In [ ]:
# Callback duration
#
# Get trace data using the provided launch file:
#    $ ros2 launch pendulum_bringup pendulum_bringup.launch.py trace:=True
#    (wait a few seconds, then kill with Ctrl+C)
#
# (optional) convert trace data:
#    $ ros2 run tracetools_analysis convert ~/.ros/tracing/pendulum/ust -o pendulum_converted
#
# OR
#
# Use the provided sample converted trace file, changing the path below to:
#    'sample_data/converted_pendulum'

In [ ]:
#path = '~/.ros/tracing/pendulum/ust'
path = 'sample_data/pendulum_converted'

In [ ]:
import sys
import datetime as dt

from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [ ]:
# Process
events = load_file(path)
handler = Ros2Handler.process(events)

In [ ]:
data_util = Ros2DataModelUtil(handler.data)

callback_symbols = data_util.get_callback_symbols()
# select timer and subscription callbacks
selected_callbacks = {key: value for (key, value) in callback_symbols.items() if 'pendulum::' in value }

output_notebook()
psize = 450
colours = ['#29788E', '#DD4968', '#410967', '#D19275']
legends = ['pendulum_driver - Timer cb',
           'pendulum_driver - Subscription cb',
           'pendulum_controller - Timer cb',
           'pendulum_controller - Subscription cb']

In [ ]:
# Plot durations separately
plot_i = 0
for obj, symbol in selected_callbacks.items():
    owner_info = data_util.get_callback_owner_info(obj)
    if owner_info is None:
        owner_info = '[unknown]'

    # Duration
    duration_df = data_util.get_callback_durations(obj)
    starttime = duration_df.loc[:, 'timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M')
    source = ColumnDataSource(duration_df)
    duration = figure(
        title=owner_info,
        x_axis_label=f'start ({starttime})',
        y_axis_label='duration (ms)',
        plot_width=psize, plot_height=psize,
    )
    duration.title.align = 'center'
    duration.line(
        x='timestamp',
        y='duration',
        legend_label=legends[plot_i],
        line_width=2,
        source=source,
        line_color=colours[plot_i],
    )
    duration.legend.label_text_font_size = '11px'
    duration.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])

    # Histogram
    dur_hist, edges = np.histogram(duration_df['duration'], bins='auto')
    duration_hist = pd.DataFrame({
        'duration': dur_hist, 
        'left': edges[:-1], 
        'right': edges[1:],
    })
    hist = figure(
        title='Duration histogram',
        x_axis_label='duration (ms)',
        y_axis_label='frequency',
        plot_width=psize, plot_height=psize,
    )
    hist.title.align = 'center'
    hist.quad(
        bottom=0,
        top=duration_hist['duration'], 
        left=duration_hist['left'],
        right=duration_hist['right'],
        fill_color=colours[plot_i],
        line_color=colours[plot_i],
    )

    plot_i += 1
    show(row(duration, hist))

In [ ]:
# Plot durations in one plot
earliest_date = None
for obj, symbol in selected_callbacks.items():
    duration_df = data_util.get_callback_durations(obj)
    thedate = duration_df.loc[:, 'timestamp'].iloc[0]
    if earliest_date is None or thedate <= earliest_date:
        earliest_date = thedate

starttime = earliest_date.strftime('%Y-%m-%d %H:%M')
duration = figure(
    title='Callback durations',
    x_axis_label=f'start ({starttime})',
    y_axis_label='duration (ms)',
    plot_width=psize, plot_height=psize,
)

plot_i = 0
for obj, symbol in selected_callbacks.items():
    duration_df = data_util.get_callback_durations(obj)
    source = ColumnDataSource(duration_df)
    duration.title.align = 'center'
    duration.line(
        x='timestamp',
        y='duration',
        legend_label=legends[plot_i],
        line_width=2,
        source=source,
        line_color=colours[plot_i],
    )
    duration.legend.label_text_font_size = '11px'

    # Print statistics
    duration_min = np.min(duration_df['duration'])
    duration_max = np.max(duration_df['duration'])
    duration_mean = np.mean(duration_df['duration'])
    duration_std = np.std(duration_df['duration'])
    
    print('callback: ', legends[plot_i])
    print("min:  {} ms".format(round(duration_min,4)))
    print("max:  {} ms".format(round(duration_max,4)))
    print("mean: {} ms".format(round(duration_mean,4)))
    print("std:  {} ms".format(round(duration_std,4)))
    
    plot_i += 1

show(duration)